### Week 6, Day 2

We're about to create and use our own MCP Server and MCP Client!

It's pretty simple, but it's not super-simple. The excitment around MCP is about how easy it is to share and use other MCP Servers - making our own does involve a bit of work.

Let's review some python code made mostly by a hard-working Engineering Team:

accounts.py

In [1]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace
from agents.mcp import MCPServerStdio
from IPython.display import display, Markdown

load_dotenv(override=True)

True

In [2]:
from accounts import Account

In [15]:
account = Account.get("Shreya")
account

Account(name='shreya', balance=10000.0, strategy='', holdings={}, transactions=[], portfolio_value_time_series=[])

In [16]:
account.buy_shares("AMZN", 3, "Because this bookstore website looks promising")

'Completed. Latest details:\n{"name": "shreya", "balance": 9906.814, "strategy": "", "holdings": {"AMZN": 3}, "transactions": [{"symbol": "AMZN", "quantity": 3, "price": 31.062, "timestamp": "2025-11-06 19:01:47", "rationale": "Because this bookstore website looks promising"}], "portfolio_value_time_series": [["2025-11-06 19:01:47", 9996.814]], "total_portfolio_value": 9996.814, "total_profit_loss": -3.1859999999996944}'

In [17]:
account.report()

'{"name": "shreya", "balance": 9906.814, "strategy": "", "holdings": {"AMZN": 3}, "transactions": [{"symbol": "AMZN", "quantity": 3, "price": 31.062, "timestamp": "2025-11-06 19:01:47", "rationale": "Because this bookstore website looks promising"}], "portfolio_value_time_series": [["2025-11-06 19:01:47", 9996.814], ["2025-11-06 19:01:51", 9933.814]], "total_portfolio_value": 9933.814, "total_profit_loss": -66.1859999999997}'

In [18]:
account.list_transactions()

[{'symbol': 'AMZN',
  'quantity': 3,
  'price': 31.062,
  'timestamp': '2025-11-06 19:01:47',
  'rationale': 'Because this bookstore website looks promising'}]

### Now we write an MCP server and use it directly!

In [19]:
# Now let's use our accounts server as an MCP server

params = {"command": "uv", "args": ["run", "accounts_server.py"]}
async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as server:
    mcp_tools = await server.list_tools()


In [20]:
mcp_tools

[Tool(name='get_balance', description='Get the cash balance of the given account name.\n\n    Args:\n        name: The name of the account holder\n    ', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_balanceArguments', 'type': 'object'}, annotations=None),
 Tool(name='get_holdings', description='Get the holdings of the given account name.\n\n    Args:\n        name: The name of the account holder\n    ', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_holdingsArguments', 'type': 'object'}, annotations=None),
 Tool(name='buy_shares', description="Buy shares of a stock.\n\n    Args:\n        name: The name of the account holder\n        symbol: The symbol of the stock\n        quantity: The quantity of shares to buy\n        rationale: The rationale for the purchase and fit with the account's strategy\n    ", inputSchema={'properties': {'name': {'title': 'Name', '

In [23]:
instructions = "You are able to manage an account for a client, and answer questions about the account."
request = "My name is Shreya and my account is under the name Shreya. What's my balance and my holdings?"
model = "gpt-4.1-mini"

In [24]:

async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as mcp_server:
    agent = Agent(name="account_manager", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("account_manager"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))


Shreya, your current cash balance is $9,906.81. You hold 3 shares of Amazon (AMZN) in your account. If you have any other questions or want to make transactions, just let me know!

### Now let's build our own MCP Client

This is not a common task and we don't really need to do it anymore because now with the `MCPServerStdio` construct you automatically get the MCP Client.

In [26]:
from accounts_client import get_accounts_tools_openai, read_accounts_resource, list_accounts_tools

mcp_tools = await list_accounts_tools() #this will spawn the server and ask it for its tools
print(mcp_tools)
openai_tools = await get_accounts_tools_openai() #reconstituted as openai function tools, we don't need to do this anymore
print(openai_tools)

[Tool(name='get_balance', description='Get the cash balance of the given account name.\n\n    Args:\n        name: The name of the account holder\n    ', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_balanceArguments', 'type': 'object'}, annotations=None), Tool(name='get_holdings', description='Get the holdings of the given account name.\n\n    Args:\n        name: The name of the account holder\n    ', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_holdingsArguments', 'type': 'object'}, annotations=None), Tool(name='buy_shares', description="Buy shares of a stock.\n\n    Args:\n        name: The name of the account holder\n        symbol: The symbol of the stock\n        quantity: The quantity of shares to buy\n        rationale: The rationale for the purchase and fit with the account's strategy\n    ", inputSchema={'properties': {'name': {'title': 'Name', 'ty

In [27]:
request = "My name is Shreya and my account is under the name Shreya. What's my balance?"

with trace("account_mcp_client"):
    agent = Agent(name="account_manager", instructions=instructions, model=model, tools=openai_tools) #when opeai_tools are called here, it will inturn call the mcp tool, which create the mcp client which launch the mcp server which inturn runs the business logic
    result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

Shreya, the cash balance in your account is $9,906.81. Is there anything else you would like to know or do with your account?

### Reading resouces does not come for free with the OPENAI SDK Package
Resources can be run as follows

In [28]:
context = await read_accounts_resource("Shreya")
print(context)

{"name": "shreya", "balance": 9906.814, "strategy": "", "holdings": {"AMZN": 3}, "transactions": [{"symbol": "AMZN", "quantity": 3, "price": 31.062, "timestamp": "2025-11-06 19:01:47", "rationale": "Because this bookstore website looks promising"}], "portfolio_value_time_series": [["2025-11-06 19:01:47", 9996.814], ["2025-11-06 19:01:51", 9933.814], ["2025-11-06 19:09:02", 10176.814], ["2025-11-06 19:23:44", 10164.814]], "total_portfolio_value": 10164.814, "total_profit_loss": 164.8140000000003}


In [ ]:
"""
#I can of course call the business logic directly in stead of the MCP logic. 
# So I would use the MCP logic if I want to share the resource so others dont need to understand my business logic.
"""
from accounts import Account 
Account.get("Shreya").report()

'{"name": "shreya", "balance": 9906.814, "strategy": "", "holdings": {"AMZN": 3}, "transactions": [{"symbol": "AMZN", "quantity": 3, "price": 31.062, "timestamp": "2025-11-06 19:01:47", "rationale": "Because this bookstore website looks promising"}], "portfolio_value_time_series": [["2025-11-06 19:01:47", 9996.814], ["2025-11-06 19:01:51", 9933.814], ["2025-11-06 19:09:02", 10176.814], ["2025-11-06 19:23:44", 10164.814], ["2025-11-06 19:24:14", 10206.814]], "total_portfolio_value": 10206.814, "total_profit_loss": 206.8140000000003}'

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/exercise.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Exercises</h2>
            <span style="color:#ff7800;">Make your own MCP Server! Make a simple function to return the current Date, and expose it as a tool so that an Agent can tell you today's date.<br/>Harder optional exercise: then make an MCP Client, and use a native OpenAI call (without the Agents SDK) to use your tool via your client.
            </span>
        </td>
    </tr>
</table>